# 信用情報の学習
# 【問題1】コンペティション内容の確認
コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。


- 何を学習し、何を予測するのか
- どのようなファイルを作りKaggleに提出するか
- 提出されたものはどういった指標値で評価されるのか

### 何を学習し、何を予測するのか
trainデータを使いmodel作成を行う  
testデータからTAAGET変数を予測する  

### どのようなファイルを作りKaggleに提出するか

IDとTARGETの2列からなるファイルを作成し、提出する

### 提出されたものはどういった指標値で評価されるのか

ROC曲線の下側(AUC)での評価となる

# 【問題2】学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。


検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。


分析の時にいい特徴量になりそうな４つを使用してモデル作成を行う

- CODE_GENDER
- NAME_FAMILY_STATUS
- AMT_CREDIT
- DAYS_BIRTH

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dataset = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
df = dataset[0:10000]

In [ ]:
df.head()

In [ ]:
y = df['TARGET'].copy()
x = df[['CODE_GENDER', 'NAME_FAMILY_STATUS', 'AMT_CREDIT', 'DAYS_BIRTH']].copy()
df_test = df[['TARGET', 'CODE_GENDER', 'NAME_FAMILY_STATUS', 'AMT_CREDIT', 'DAYS_BIRTH']].copy()

In [ ]:
df_test.info()

##  ダミー変数作成

In [ ]:
x['CODE_GENDER'] 

In [ ]:
x['CODE_GENDER'] = x['CODE_GENDER'].map({'M' : 0, 'F' : 1}).astype(int) # 性別のダミー変数
x['NAME_FAMILY_STATUS'] = x['NAME_FAMILY_STATUS'].map({'Civil marriage' : 0, 'Married' : 1, 'Separated' : 2, 'Single / not married' : 3, 'Widow' : 4}).astype(int) # 性別のダミー変数

In [ ]:
x.head()

## 相関図

In [ ]:
import seaborn as sns
sns.pairplot(df_test);

##  データの切り分け

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(x.values, random_state=0).copy()
y_train, y_true = train_test_split(y.values, random_state=0).copy()

## ベースラインモデル作成

In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as me
from sklearn.metrics import classification_report

logi = LogisticRegression()
logi.fit(x_train, y_train)
logi_pred = logi.predict(x_test)
logi_confusion = me.confusion_matrix(y_true, logi_pred) 
print(logi_confusion) # 混同行列
print(classification_report(y_true, logi_pred))

In [ ]:
from sklearn.metrics import roc_auc_score
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi.predict_proba(x_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi.predict_proba(x_test)[:, 1])))

# 【問題4】特徴量エンジニアリング
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。


どの特徴量を使うか
どう前処理をするか

何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。

## 学習方法の判断
### 近傍法

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh5 = KNeighborsClassifier(n_neighbors=5)
neigh5.fit(x_train, y_train) # 機械に学ばせている
KNN5_pred = neigh5.predict(x_test) # 検証データを渡して、結果を得る
print(np.sum(KNN5_pred==0)) # pythonではTrueを1としてあつかうため合計値でいける
print(np.sum(KNN5_pred==1))

結果をみると0の割合が高すぎるのか1を表示できなかった。  
k近傍法は目的変数の数が同じじゃないと使いづらい？？？  
少しチューニング

In [ ]:
neigh3 = KNeighborsClassifier(n_neighbors=3)
neigh3.fit(x_train, y_train) # 機械に学ばせている
KNN3_pred = neigh3.predict(x_test) # 検証データを渡して、結果を得る
print(np.sum(KNN3_pred==0))
print(np.sum(KNN3_pred==1))

In [ ]:
neigh1 = KNeighborsClassifier(n_neighbors=1)
neigh1.fit(x_train, y_train) # 機械に学ばせている
KNN1_pred = neigh1.predict(x_test) # 検証データを渡して、結果を得る
print(np.sum(KNN1_pred==0))
print(np.sum(KNN1_pred==1))

In [ ]:
import sklearn.metrics as me
from sklearn.metrics import classification_report
knn5_confusion = me.confusion_matrix(y_true, KNN5_pred) 
print(knn5_confusion) # 混同行列
print(classification_report(y_true, KNN5_pred))

In [ ]:
knn3_confusion = me.confusion_matrix(y_true, KNN3_pred) 
print(knn3_confusion) # 混同行列
print(classification_report(y_true, KNN3_pred))

In [ ]:
knn1_confusion = me.confusion_matrix(y_true, KNN1_pred) 
print(knn1_confusion) # 混同行列
print(classification_report(y_true, KNN1_pred))

正解率は高いが、正解ラベル判断精度はかなり低い  
正解ラベルの特徴をうまく抜き出せていない証拠

In [ ]:
unique_name, count = np.unique(neigh3.predict_proba(x_test)[:, 1], return_counts=True) # 閾値とそれを超える値を格納
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'])

0.6時点ではかなり生き残ってる

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, neigh3.predict_proba(x_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, neigh3.predict_proba(x_test)[:, 1])))

### ランダムフォレスト

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf_pred = clf.predict(x_test)
clf_confusion = me.confusion_matrix(y_true, clf_pred) 
print(clf_confusion) # 混同行列
print(classification_report(y_true, clf_pred))

In [ ]:
unique_name, count = np.unique(clf.predict_proba(x_test)[:, 1], return_counts=True) # 正解ラベル１を取る確率関数
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'])

In [ ]:
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, clf.predict_proba(x_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, clf.predict_proba(x_test)[:, 1])))

ランダムフォレストの場合デフォルトでサンプリングを100回行うので確率分布はかなり細かくなる。  
そのパラメータを変更すると...

In [ ]:
clf_1000 = RandomForestClassifier(n_estimators=1000)
clf_1000.fit(x_train, y_train)
clf_1000_pred = clf_1000.predict(x_test)
clf_1000_confusion = me.confusion_matrix(y_true, clf_1000_pred) 
print(clf_1000_confusion) # 混同行列
print(classification_report(y_true, clf_1000_pred))

In [ ]:
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, clf_1000.predict_proba(x_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, clf_1000.predict_proba(x_test)[:, 1])))

おおむね予想通りの結果出力となった。

### 決定木

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
dtc_pred = dtc.predict(x_test)
dtc_confusion = me.confusion_matrix(y_true, dtc_pred) 
print(dtc_confusion) # 混同行列
print(classification_report(y_true, dtc_pred))

In [ ]:
unique_name, count = np.unique(dtc.predict_proba(x_test)[:, 1], return_counts=True) # 閾値とそれを超える値を格納
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'])

おもってた結果には全くならなかった  
0.5で2個1で190個ってどういう意味なんだろうか  
そしたら190がTPで出力されそうなものだが  
決定木の場合閾値で判断してない説

In [ ]:
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, dtc.predict_proba(x_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, dtc.predict_proba(x_test)[:, 1])))

### SVC

In [ ]:
from sklearn.svm import SVC
svc = SVC(probability=True) # probabilityで確率出力
svc.fit(x_train, y_train)
svc_pred = svc.predict(x_test)
svc_confusion = me.confusion_matrix(y_true, svc_pred) 
print(svc_confusion) # 混同行列
print(classification_report(y_true, svc_pred))

In [ ]:
unique_name, count = np.unique(svc.predict_proba(x_test)[:, 1], return_counts=True) # 閾値とそれを超える値を格納
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'])

各データに対して　1を取る確率

In [ ]:
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, svc.predict_proba(x_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, svc.predict_proba(x_test)[:, 1])))

ひとつも正解ラベル見つけられてないという悲しき結果に  
閾値に関してはユニークが2500個(データの数ごとに確率だしてくれてる)のに362しか出力されていないし  
AUCに関してはおそらくだけど各データに対して0を推奨しているため、0.5より下に行ったんじゃないかなぁ

### ロジスティック回帰　

In [ ]:
from sklearn.linear_model import LogisticRegression
logi = LogisticRegression()
logi.fit(x_train, y_train)
logi_pred = svc.predict(x_test)
logi_confusion = me.confusion_matrix(y_true, logi_pred) 
print(logi_confusion) # 混同行列
print(classification_report(y_true, logi_pred))

In [ ]:
unique_name, count = np.unique(logi.predict_proba(x_test)[:, 1], return_counts=True) # 閾値とそれを超える値を格納
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'])

In [ ]:
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi.predict_proba(x_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi.predict_proba(x_test)[:, 1])))

かなりスコアは良いかもしれない  
理由はおいおいわかりそうだが...

### 評価
ロジスティック回帰1強  
今後の学習はロジスティック回帰を使っていく

## 対数変換

In [ ]:
x_l = x.copy()

In [ ]:
x_l['AMT_CREDIT_log'] = x_l['AMT_CREDIT'].apply(np.log) # 対数変換
sns.displot(x_l.AMT_CREDIT_log);

In [ ]:
x_l.head()

In [ ]:
x_log = x_l.drop('AMT_CREDIT', axis=1)
x_log.head()

### 評価

In [ ]:
x_train_l, x_test_l = train_test_split(x_log.values, random_state=0).copy()

In [ ]:
logi_l = LogisticRegression()
logi_l.fit(x_train_l, y_train)
logi_l_pred = logi_l.predict(x_test_l)
logi_l_confusion = me.confusion_matrix(y_true, logi_l_pred) 
print(logi_l_confusion) # 混同行列
print(classification_report(y_true, logi_l_pred))

In [ ]:
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi_l.predict_proba(x_test_l)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi_l.predict_proba(x_test_l)[:, 1])))

対数変換した方が結果は悪くなった。  
住宅価格の時は良くなったので考えられる原因は
- 対数変換したのが、目的変数ではなく説明変数のみ
- 分類問題だったため

## 標準化

In [ ]:
x_train_df, x_test_df = train_test_split(x, random_state=0).copy() # df状態で分配

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train_df[['AMT_CREDIT', 'DAYS_BIRTH']])
x_train_h = scaler.transform(x_train_df[['AMT_CREDIT', 'DAYS_BIRTH']]) # 標準化できるものを標準化しhに格納
x_test_h = scaler.transform(x_test_df[['AMT_CREDIT', 'DAYS_BIRTH']])
x_train_h

In [ ]:
x_train_df_2 = x_train_df.drop(['AMT_CREDIT', 'DAYS_BIRTH'], axis=1) # 標準化したものを削除
x_test_df_2 = x_test_df.drop(['AMT_CREDIT', 'DAYS_BIRTH'], axis=1) 
x_train_df_2

In [ ]:
x_train_2 = x_train_df_2.values # dfをndarrayに
x_test_2 = x_test_df_2.values
x_train_h_2 = np.hstack([x_train_h, x_train_2])
x_test_h_2 = np.hstack([x_test_h, x_test_2])
x_test_h_2

### 評価

In [ ]:
logi_h = LogisticRegression()
logi_h.fit(x_train_h_2, y_train)
logi_h_pred = logi_h.predict(x_test_h_2)
logi_h_confusion = me.confusion_matrix(y_true, logi_h_pred) 
print(logi_h_confusion) # 混同行列
print(classification_report(y_true, logi_h_pred))

In [ ]:
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi_h.predict_proba(x_test_h_2)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi_h.predict_proba(x_test_h_2)[:, 1])))

こちらも精度としては下がってる  
何でもかんでも標準化すればいいってわけではない  
可能性としては、'AMT_CREDIT', 'DAYS_BIRTH'の重みが深く、標準化したことで重みが減ってしまったこと。

## 特徴量の削減

- CODE_GENDER = c
- NAME_FAMILY_STATUS = n
- AMT_CREDIT = a
- DAYS_BIRTH = d

aが一番関係なさそうなので削ってみる

In [ ]:
x_cnd = x.drop('AMT_CREDIT', axis=1)
x_train_cnd, x_test_cnd = train_test_split(x_cnd.values, random_state=0).copy()

### 評価

In [ ]:
logi_cnd = LogisticRegression()
logi_cnd.fit(x_train_cnd, y_train)
logi_cnd_pred = logi_cnd.predict(x_test_cnd)
logi_cnd_confusion = me.confusion_matrix(y_true, logi_cnd_pred) 
print(logi_cnd_confusion) # 混同行列
print(classification_report(y_true, logi_cnd_pred))
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi_cnd.predict_proba(x_test_cnd)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi_cnd.predict_proba(x_test_cnd)[:, 1])))

特徴量を削減することでスコアは若干下がった  
追加すれば上がるかも！　　



ちなみに他のデータを削ると

In [ ]:
x_nad = x.drop('CODE_GENDER', axis=1)
x_train_nad, x_test_nad = train_test_split(x_nad.values, random_state=0).copy()

In [ ]:
logi_nad = LogisticRegression()
logi_nad.fit(x_train_nad, y_train)
logi_nad_pred = logi_nad.predict(x_test_nad)
logi_nad_confusion = me.confusion_matrix(y_true, logi_nad_pred) 
print(logi_nad_confusion) # 混同行列
print(classification_report(y_true, logi_nad_pred))
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi_nad.predict_proba(x_test_nad)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi_nad.predict_proba(x_test_nad)[:, 1])))

genderを削除してもスコアは下がった！

In [ ]:
x_gad = x.drop('NAME_FAMILY_STATUS', axis=1)
x_train_gad, x_test_gad = train_test_split(x_gad.values, random_state=0).copy()

In [ ]:
logi_gad = LogisticRegression()
logi_gad.fit(x_train_gad, y_train)
logi_gad_pred = logi_gad.predict(x_test_gad)
logi_gad_confusion = me.confusion_matrix(y_true, logi_gad_pred) 
print(logi_gad_confusion) # 混同行列
print(classification_report(y_true, logi_gad_pred))
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi_gad.predict_proba(x_test_gad)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi_gad.predict_proba(x_test_gad)[:, 1])))

genderを削った時と全く同じスコアになった。  
ということは全く同じ重みになってる？  
あるいはカテゴリー系の重みは一定なのか？

In [ ]:
x_gan = x.drop('DAYS_BIRTH', axis=1)
x_train_gan, x_test_gan = train_test_split(x_gan.values, random_state=0).copy()

In [ ]:
logi_gan = LogisticRegression()
logi_gan.fit(x_train_gan, y_train)
logi_gan_pred = logi_gan.predict(x_test_gan)
logi_gan_confusion = me.confusion_matrix(y_true, logi_gan_pred) 
print(logi_gan_confusion) # 混同行列
print(classification_report(y_true, logi_gan_pred))
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi_gan.predict_proba(x_test_gan)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi_gan.predict_proba(x_test_gan)[:, 1])))

ものすごく下がった。4パターン全て行いわかったのが重要度は
1. DAYS_BIRTH 
2. AMT_CREDIT 
3. CODE_GENDER = NAME_FAMILY_STATUS 
  
の順番だった！

## 特徴量の追加
次に良さそうだったAMT_INCOME_TOTALを追加

In [ ]:
x2 = df[['CODE_GENDER', 'NAME_FAMILY_STATUS', 'AMT_CREDIT', 'DAYS_BIRTH', 'AMT_INCOME_TOTAL']].copy()
x2['CODE_GENDER'] = x2['CODE_GENDER'].map({'M' : 0, 'F' : 1}).astype(int) # 性別のダミー変数
x2['NAME_FAMILY_STATUS'] = x2['NAME_FAMILY_STATUS'].map({'Civil marriage' : 0, 'Married' : 1, 'Separated' : 2, 'Single / not married' : 3, 'Widow' : 4}).astype(int) # 性別のダミー変数
x2_train, x2_test = train_test_split(x2.values, random_state=0).copy()

### 評価

In [ ]:
logi2 = LogisticRegression()
logi2.fit(x2_train, y_train)
logi2_pred = logi2.predict(x2_test)
logi2_confusion = me.confusion_matrix(y_true, logi2_pred) 
print(logi2_confusion)
print(classification_report(y_true, logi2_pred))
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi2.predict_proba(x2_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi2.predict_proba(x2_test)[:, 1])))

がっつり下がった笑

'DAYS_REGISTRATION'の場合

In [ ]:
x3 = df[['CODE_GENDER', 'NAME_FAMILY_STATUS', 'AMT_CREDIT', 'DAYS_BIRTH', 'DAYS_REGISTRATION']].copy()
x3['CODE_GENDER'] = x3['CODE_GENDER'].map({'M' : 0, 'F' : 1}).astype(int) # 性別のダミー変数
x3['NAME_FAMILY_STATUS'] = x3['NAME_FAMILY_STATUS'].map({'Civil marriage' : 0, 'Married' : 1, 'Separated' : 2, 'Single / not married' : 3, 'Widow' : 4}).astype(int) # 性別のダミー変数
x3_train, x3_test = train_test_split(x3.values, random_state=0).copy()

In [ ]:
logi3 = LogisticRegression()
logi3.fit(x3_train, y_train)
logi3_pred = logi3.predict(x3_test)
logi3_confusion = me.confusion_matrix(y_true, logi3_pred) 
print(logi3_confusion)
print(classification_report(y_true, logi3_pred))
fpr_all, tpr_all, th_all  = me.roc_curve(y_true, logi3.predict_proba(x3_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(y_true, logi3.predict_proba(x3_test)[:, 1])))

ほんとに若干だが上がった！  
関係はあったみたい！

## 全データでの検証

In [ ]:
dff = dataset
yf = dff['TARGET'].copy()
xf = dff[['CODE_GENDER', 'NAME_FAMILY_STATUS', 'AMT_CREDIT', 'DAYS_BIRTH', 'DAYS_REGISTRATION']].copy()

In [ ]:
unique_name, count = np.unique(xf.CODE_GENDER, return_counts=True) # 閾値とそれを超える値を格納
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'], index=dff.CODE_GENDER.unique())

In [ ]:
unique_name, count = np.unique(xf.NAME_FAMILY_STATUS, return_counts=True) # 閾値とそれを超える値を格納
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'], index=dff.NAME_FAMILY_STATUS.unique())

In [ ]:
xf['CODE_GENDER'] = xf['CODE_GENDER'].map({'M' : 0, 'F' : 1, 'XNA' : 2}).astype(int) # 性別のダミー変数
xf['NAME_FAMILY_STATUS'] = xf['NAME_FAMILY_STATUS'].map({'Civil marriage' : 0, 'Married' : 1, 'Separated' : 2, 'Single / not married' : 3, 'Widow' : 4, 'Unknown' : 5}).astype(int) # 性別のダミー変数

In [ ]:
xf_train, xf_test = train_test_split(xf.values, random_state=0).copy()
yf_train, yf_true = train_test_split(yf.values, random_state=0).copy()

### 評価

In [ ]:
logi_f = LogisticRegression()
logi_f.fit(xf_train, yf_train)
logi_f_pred = logi_f.predict(xf_test)
logi_f_confusion = me.confusion_matrix(yf_true, logi_f_pred) 
print(logi_f_confusion)
print(classification_report(yf_true, logi_f_pred))
fpr_all, tpr_all, th_all  = me.roc_curve(yf_true, logi_f.predict_proba(xf_test)[:, 1], pos_label=1)# 正解ラベル１を1に設定、なのでみるべきなのは0を取る時の確率密度
df_roc = pd.DataFrame({'th_all': th_all, 'tpr_all': tpr_all, 'fpr_all': fpr_all}) # 閾値、再現率(recall)、陰性を間違えて陽性と判断した場合
plt.plot(fpr_all, tpr_all, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()
display(df_roc)
print('AUC : {}'.format(roc_auc_score(yf_true, logi_f.predict_proba(xf_test)[:, 1])))

信じられないくらい精度は落ちた  
実行時間はそこまでかからなかった.  
外れ値の問題かサンプル時のかたより(10000も引っ張ったしそれはないと思うけど)の可能性が高い  
過学習の可能性もある

## テストデータでの実行

In [ ]:
testdata = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
x_testdata = testdata[['CODE_GENDER', 'NAME_FAMILY_STATUS', 'AMT_CREDIT', 'DAYS_BIRTH', 'DAYS_REGISTRATION']].copy() # データの読み込み

In [ ]:
x_testdata['CODE_GENDER'] = x_testdata['CODE_GENDER'].map({'M' : 0, 'F' : 1, 'XNA' : 2}).astype(int) # 性別のダミー変数
x_testdata['NAME_FAMILY_STATUS'] = x_testdata['NAME_FAMILY_STATUS'].map({'Civil marriage' : 0, 'Married' : 1, 'Separated' : 2, 'Single / not married' : 3, 'Widow' : 4, 'Unknown' : 5}).astype(int) # 性別のダミー変数

In [ ]:
logi_pred_testdata = logi_f.predict(x_testdata.values) # 結果の出力

In [ ]:
unique_name, count = np.unique(logi_pred_testdata, return_counts=True) # 閾値とそれを超える値を格納
pd.DataFrame(list(zip(unique_name, count)), columns = ['unique_name','count'])

案の定確率で出さない限りは0で出力される。

In [ ]:
logi_pred_proba_testdata = logi_f.predict_proba(x_testdata.values) # 確率密度での結果の出力

In [ ]:
result = pd.DataFrame(logi_f.predict_proba(x_testdata)[:, 1], columns=['TARGET']) # dfにして保存

In [ ]:
x_testdata_id = testdata[['SK_ID_CURR']].copy() # idの列の抽出

In [ ]:
result_1 = pd.concat([x_testdata_id, result], axis=1)
RESULT = result_1.set_index('SK_ID_CURR') # 結合とindexの振り直し

In [ ]:
RESULT.to_csv('result.csv')